# Feature engineering : Prévision de production solaire à J+1 en temps réel

***Résumé Exécutif de l'Exploration de Données (EDA) - Prévision Solaire J+1***

Cette analyse exploratoire des données (EDA) vise à comprendre les caractéristiques de la production solaire régionale et des variables météorologiques associées, en préparation du développement d'un modèle de prévision à J+1.

**Données** : Dataset horaire sur 2.5 ans (01/2023 - 05/2025) de production RTE et prévisions OpenMétéo. Identification de 2 valeurs manquantes nocturnes dans la cible, qui seront imputées à 0.

**Variable Cible ("solar_mw")** :
- Forte saisonnalité journalière et annuelle, avec une dérive positive due à l'augmentation de la capacité installée ;
- Distribution asymétrique à droite, avec de nombreux zéros (nuit/mauvaises conditions) ; 
- Nécessite une différenciation saisonnière d'ordre 24 pour la stationnarité (confirmée par test ADFuller), avec un ordre AR de 1 (analyse PACF), et un ordre MA de q (à déterminer).

**Variables Explicatives (Covariables) :**
- Fortes corrélations (+0.8) avec la cible : global_tilted_irradiance, shortwave_radiation ;
- Redondances identifiées : Irradiance/Radiation (choix de l'irradiance pour le sens physique), température/température apparente (choix de temperature_2m) ;
- Variables de dispersion (delta_minmax, std) : Essentielles pour capturer l'hétérogénéité spatiale de la région, notamment pour la couverture nuageuse et la DNI ;
- Cross-corrélation : Déphasages clairs pour l'irradiance, l'humidité, la vitesse du vent et la température (justifiant des lags aux ordres 24, 36, 48...). Pression et couverture nuageuse non pertinentes pour les lags.

**Prochaines Étapes (déjà identifiées)**:
- Feature Engineering (encodage cyclique, features laggées, agrégations) ;
- Baseline SARIMAX avec les features physiques les plus corrélées.
- Sélection de features (RFE ou Embedding via LightGBM) ;
- Validation croisée "Expanding Window" avec optimisation des hyperparamètres (Optuna) ;
- Développement de modèles LightGBM et LSTM, avec MC dropout et regression quantile pour quantifier l'incertitude des modèles

### Import des libraries